## Efficient coding of natural images with a population of noisy Linear-Nonlinear neurons

The following is an implementation of Karklin & Simoncelli's 2011 paper, with a model of the visual system maximizing mutual information, minimizing spiking with natural images as input. It is implemented in Tensorflow.

### Model:
![fig1](./img/fig1a.png)


### Response Function:
$$ r_j = f_j(y_j)+n_r $$  

$$ y_j = \textbf{w}_j^T(x+n_x) $$

Add indepenent (for now - Pratik will explore non iid) noise in our response, also add noise in our input image, before calcuating non-linear function.

### Objective Function:
$$ I(X;R) - \sum_j \lambda_j \langle r_j \rangle $$  

Maximize mutual information between image and response, minimizing spiking rate of all neurons. $\lambda$ as the tradeoff between the two.

In [1]:
#dependencies

import h5py
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
sess = tf.InteractiveSession()

%matplotlib inline

In [2]:
def extract_images(filename):
    #function from Dylan
    with h5py.File(filename, "r") as f:
        full_img_data = np.array(f['van_hateren_good'], dtype=np.float32)
    return full_img_data

class vanHateren:
    def __init__(self,
                 img_dir,
                 patch_edge_size=None,
                 normalize=False,
                 rand_state=np.random.RandomState()):
        self.images = self.extract_images(img_dir, patch_edge_size, normalize)

    """
    adapted from Dylan Payton's code for Sparse coding here: https://github.com/dpaiton/FeedbackLCA/blob/master/data/input_data.py
    load in van hateren dataset
    if patch_edge_size is specified, rebuild data array to be of sequential image
    patches.
    if preprocess is true, subtract mean from each full-size image, and rescale image variance to 1
    Note: in K&S2011, methods report input images' piel values were 'linear with respect to light intensity'
    I'm not sure if this is true for the VH images we are using, and how to properly normalize for this if not.
    """

    def extract_images(self, filename, patch_edge_size=None, normalize=False):
        with h5py.File(filename, "r") as f:
            full_img_data = np.array(f['van_hateren_good'], dtype=np.float32)
            if(normalize):
                print('normalizing...')
                full_img_data = full_img_data - np.mean(full_img_data,axis=(1,2),keepdims=True)
                full_img_data = full_img_data/np.std(full_img_data,axis=(1,2),keepdims=True)
            if patch_edge_size is not None:
                print('sectioning into patches....')
                (num_img, num_px_rows, num_px_cols) = full_img_data.shape
                num_img_px = num_px_rows * num_px_cols
                assert np.sqrt(num_img_px) % patch_edge_size == 0, ("The number of image edge pixels % the patch edge size must be 0.")
                self.num_patches = int(num_img_px / patch_edge_size**2)
                full_img_data = np.reshape(full_img_data, (num_img, num_img_px))
                data = np.vstack([full_img_data[idx,...].reshape(self.num_patches, patch_edge_size, patch_edge_size) for idx in range(num_img)])
            else:
                data = full_img_data
                self.num_patches = 0
            return data


In [90]:
# Model Parameters
nneurons = 100
patchsize = 16


#noise - these Prateek is interested in changing to removed iid asumption
noisexsigma = 0.4
noisersigma = 2

lambdaj = 0.1 #this is adjusted in order to get ravg = 1
ravg = 1

batchsize = 100
iterations = 1e6

In [91]:
#full images
#vhims = extract_images('../vanHaterenNaturalImages/VanHaterenNaturalImagesCurated.h5')

#image patches (as in Karklin& Simoncelli)
vhims = vanHateren(
    img_dir='../vanHaterenNaturalImages/VanHaterenNaturalImagesCurated.h5',
    normalize = True,
    patch_edge_size=patchsize
    )


normalizing...
sectioning into patches....


In [1]:
#example image
plt.imshow(vhims.images[400,:,:],cmap='gray', interpolation='None')

NameError: name 'plt' is not defined

In [93]:
#inputs & outputs
x = tf.placeholder(tf.float32, shape=[patchsize**2])

#weights and reul
W = tf.Variable(tf.zeros([patchsize**2,nneurons]))
reluslope = tf.Variable(tf.zeros([nneurons]))
reluoff = tf.Variable(tf.zeros([nneurons]))

#b = tf.Variable(tf.zeros([nneurons]))

#initialize vairaibles
#sess.run(tf.initialize_all_variables())

noisex = np.random.normal(loc=0, scale=noisexsigma**2, size=patchsize**2)
noiser = np.random.normal(loc=0, scale=noisersigma**2, size=nneurons)

# y = weights * (x + noise_x)
# r = reluslope * sigmoid(y+reluoff)  + noise_r

y = tf.matmul(tf.add(x,noisex),W)
r = tf.add(reluslope*tf.nn.relu(y + reluoff), noiser)

#objective (maximize): I(X,R)-sum_j(lambda*<r_j>)
mask = tf.cast(tf.less(np.zeros_like(y)+reluoff,y),tf.float32)
G = tf.diag(tf.mul(reluslope,mask))[:,:,1,:] #want size: [batches,100,100]

cx = np.dot(tf.transpose(x), x)
cnr = np.dot(noiser,tf.transpose(noiser))
cnx = np.dot(noisex,tf.transpose(noisex))


#crx = G @ W.T @ np.cov(noisex) @ W @ G+ np.cov(noiser)
print(W)
print(G)
test = tf.matmul(W,G)
    
cxr = tf.matrix_inverse(tf.matrix_inverse(cx) + 
                        tf.matmul(W, tf.matmul(G, 
                                         tf.matmul(tf.matrix_inverse(tf.matmul(G,
                                                                        tf.matmul(tf.transpose(W), 
                                                                                   tf.matmul(cnx, 
                                                                                           tf.matmul(W, G)
                                                                                         ))) 
                                                                         + cnr), tf.matmul(G, tf.transpose(W))))))
information = tf.mean(0.5*2 *np.log(2*np.pi*e*tf.matrix_determinant(cxr)))
objective =  information - tf.mean(r)/nneurons

optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(-1*objective)





Tensor("Squeeze_7:0", shape=(?, 100, 100), dtype=float32)


ValueError: Shape (?, 100, 100) must have rank 2

In [73]:
print(W)

In [63]:
#define variables
init = tf.initialize_all_variables()
 
#define & launch session
sess = tf.Session()
sess.run(init)

#train for n iterations
for step in xrange(iterations):
    sess.run(train_step, feed_dict = {x: images_feed})
    